In [1]:
import networkx as nx
import numpy as np
import pandas as pd

In [2]:
def dag_to_dist_matrix(dag, observables, cards, seed):
    np.random.seed(seed)
    n_simulations = 1000
    hiddens = [node for node in dag.nodes if node not in observables]
    
    counts_matrix = np.zeros(cards)
    
    for _ in range(n_simulations):
        hidden_vals = {h: np.random.randint(2) for h in hiddens}
        
        # track observable values
        current_vals = []
        for node in observables:
            parents = list(dag.predecessors(node))
            if parents:
                parent_vals = np.array([hidden_vals[parent] if parent in hiddens else current_vals[observables.index(parent)] for parent in parents])
                weights = np.random.dirichlet(np.ones(len(parents)))  # Weights sum to 1
                weighted_sum = np.dot(parent_vals, weights)
                node_val = 1 if weighted_sum > 0.5 else 0 # logit func
            else:
                node_val = np.random.randint(2)
            current_vals.append(node_val)
        
        # increment the count
        counts_matrix[tuple(current_vals)] += 1

    # normalization
    print(counts_matrix)
    probability_matrix = counts_matrix / n_simulations
    return probability_matrix

In [3]:
def dag_to_dist_stronger_AB(dag, observables, cards, seed):
    np.random.seed(seed)
    n_simulations = 1000
    hiddens = [node for node in dag.nodes if node not in observables]
    counts_matrix = np.zeros(cards)
    
    for _ in range(n_simulations):
        hidden_vals = {h: np.random.randint(2) for h in hiddens}
        current_vals = []
        
        for node in observables:
            parents = list(dag.predecessors(node))
            if parents:
                parent_vals = np.array([hidden_vals[parent] if parent in hiddens else current_vals[observables.index(parent)] for parent in parents])
                if node == 'B':
                    # A->B strong causal arrow strength
                    dirichlet_params = np.ones(len(parents))
                    dirichlet_params[-1] *= 5
                else:
                    dirichlet_params = np.ones(len(parents))
                weights = np.random.dirichlet(dirichlet_params)
                weighted_sum = np.dot(parent_vals, weights)
                node_val = 1 if weighted_sum > 0.5 else 0
            else:
                node_val = np.random.randint(2)
            current_vals.append(node_val)
        
        counts_matrix[tuple(current_vals)] += 1
    probability_matrix = counts_matrix / n_simulations
    
    return probability_matrix

In [50]:
G = nx.DiGraph()
G.add_edges_from([('h1', 'A'), ('h1', 'C'), ('h2', 'A'), ('h2', 'B'), ('h3', 'B'), ('h3', 'C'), ('A', 'B')])
dag, observables, cards, seed = G, ['A', 'B', 'C'], (2, 2, 2), 18 

np.random.seed(seed)
n_simulations = 1000
hiddens = [node for node in dag.nodes if node not in observables]
counts_matrix = np.zeros(cards)

for _ in range(n_simulations):
    hidden_vals = {h: np.random.randint(2) for h in hiddens}
    current_vals = []
    
    for node in observables:
        parents = list(dag.predecessors(node))
        if parents:
            parent_vals = np.array([hidden_vals[parent] if parent in hiddens else current_vals[observables.index(parent)] for parent in parents])
            if node == 'B' or node == 'A':
                # A->B strong causal arrow strength
                dirichlet_params = np.ones(len(parents))
                dirichlet_params[-1] *= 14
            else:
                dirichlet_params = np.ones(len(parents))
            weights = np.random.dirichlet(dirichlet_params)
            weighted_sum = np.dot(parent_vals, weights)
            node_val = 1 if weighted_sum > 0.5 else 0
        else:
            node_val = 0
        current_vals.append(node_val)
    
    counts_matrix[tuple(current_vals)] += 1
probability_matrix = counts_matrix / n_simulations
probability_matrix

array([[[0.254, 0.24 ],
        [0.   , 0.   ]],

       [[0.   , 0.   ],
        [0.259, 0.247]]])

In [42]:
dirichlet_params = np.ones(len(parents))
dirichlet_params[-1] *= 5
np.random.dirichlet(dirichlet_params)

array([0.05085855, 0.94914145])

In [4]:
G = nx.DiGraph()
G.add_edges_from([('h1', 'A'), ('h1', 'C'), ('h2', 'A'), ('h2', 'B'), ('h3', 'B'), ('h3', 'C'), ('A', 'B')])
dag_to_dist_stronger_AB(G, 
                   ['A', 'B', 'C'], 
                   (2, 2, 2), 
                   18)

array([[[0.289, 0.179],
        [0.007, 0.008]],

       [[0.008, 0.005],
        [0.179, 0.325]]])

In [5]:
def simulate_B_given_do_A(A_value, seed):
    """
    Simulate B given do(A=A_value) a certain number of times and return the probability of B=1.
    """
    np.random.seed(seed)
    B_values = []
    n_simulations = 10000
    for _ in range(n_simulations):
        h3 = np.random.randint(2)  # Simulate hidden variable
        parents = [h3, A_value]
        dirichlet_params = np.ones(len(parents))
        dirichlet_params[-1] *= 5  # A's influence is stronger
        weights = np.random.dirichlet(dirichlet_params)
        weighted_sum = np.dot(parents, weights)
        B_val = 1 if weighted_sum > 0.0001 else 0
        B_values.append(B_val)
    
    return np.mean(B_values)

# Calculate P(B=1|do(A=0)) and P(B=1|do(A=1))
prob_B_given_do_A0 = simulate_B_given_do_A(0, 13)
prob_B_given_do_A1 = simulate_B_given_do_A(1, 31)

prob_B_given_do_A1-prob_B_given_do_A0

0.5021